In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import sys, os
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('../util'))
%matplotlib inline

In [172]:
data_path = Path('../data/')
# data_genre = data_path / Path('data_w_genres.csv')
df_path= data_path / Path('data_split_by_artist.csv')

CSV file contains float columns as features and using artist as a label for multiclass prediction.
If a song was produced by >1 artist the row has been split into mulitple rows so that each row only
has one artist.

In [190]:
df = pd.read_csv(df_path,index_col=0)
df.head()

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,artists
0,0.995,0.708,158648,0.1950,0,0.563,10,0.1510,-12.428,1,0,0.0506,118.469,0.7790,Carl Woitschach
1,0.994,0.379,282133,0.0135,0,0.901,8,0.0763,-28.454,1,0,0.0462,83.972,0.0767,Robert Schumann
2,0.994,0.379,282133,0.0135,0,0.901,8,0.0763,-28.454,1,0,0.0462,83.972,0.0767,Vladimir Horowitz
3,0.604,0.749,104300,0.2200,0,0.000,5,0.1190,-19.924,0,0,0.9290,107.177,0.8800,Seweryn Goszczyński
4,0.995,0.781,180760,0.1300,0,0.887,1,0.1110,-14.734,0,0,0.0926,108.003,0.7200,Francisco Canaro


In [191]:
df_new.artists.value_counts()

Francisco Canaro           2234
Эрнест Хемингуэй           1215
Frédéric Chopin            1033
Ludwig van Beethoven        976
Wolfgang Amadeus Mozart     938
                           ... 
 Gauri Bapat                  1
Bic Runga                     1
C. S. Radhadevi               1
 Carlos Ponce                 1
                              1
Name: artists, Length: 31466, dtype: int64